In [2]:
conda install -c conda-forge transformers

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\sauar\anaconda3

  added / updated specs:
    - transformers


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    huggingface_hub-0.11.1     |     pyhd8ed1ab_0         127 KB  conda-forge
    tokenizers-0.11.4          |   py39he5181cf_1         2.4 MB
    transformers-4.24.0        |   py39haa95532_0         4.7 MB
    ------------------------------------------------------------
                                           Total:         7.3 MB

The following NEW packages will be INSTALLED:

  huggingface_hub    conda-forge/noarch::huggingface_hub-0.11.1-pyhd8ed1ab_0 
  tokenizers         pkgs/main/win-64::tokenizers-0.11.4-py39he5181cf_1 
  transformers       pkgs/main/win-64::transformers-4.24.0-py39haa95532_0 




transformers-4.24.0  | 4.7 MB    |            |   0% 

tokenizers-0.11.4    | 2.4 M

In [1]:

from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
import torch
from PIL import Image

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cpu")
model.to(device)



max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
def predict_step(image_paths):
    images = []
    for image_path in image_paths:
        i_image = Image.open(image_path)
        if i_image.mode != "RGB":
            i_image = i_image.convert(mode="RGB")

        images.append(i_image)

    pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    output_ids = model.generate(pixel_values, **gen_kwargs)

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]
    return preds


predict_step(['gun.jpg']) # ['a woman in a hospital bed with a woman in a hospital bed']


['a man in a black jacket carrying a black backpack']

In [14]:
import time
start = time.time()

In [13]:

predict_step(['dogwater.png']) # ['a woman in a hospital bed with a woman in a hospital bed']


['a dog and a cat playing in the water']

In [3]:
import nltk

In [53]:
s = ['a black and white photo man with gun', 'a person is with a gun', 'a black and white photo of a gun', 'a women riding a dirt bike on gun', 'soldier with a gun']
h = 'a man in a suit holding a cell phone'


In [54]:
print('GPT2 generated caption BLEU Score',nltk.translate.bleu_score.sentence_bleu(s,h,weights = (0.5,0.5)))

GPT2 generated caption BLEU Score 0.7590721152765896


In [55]:
k = 'a man holding a gun' 
print('Our model generated caption BLEU Score',nltk.translate.bleu_score.sentence_bleu( s,k,weights = (0.5,0.5)))

Our model generated caption BLEU Score 0.97182531580755


In [52]:
# d = 'a black and brown dog playing in the water at a beach'
d = ['a black and brown dog playing in the water at a beach', 'a black dog and a dog that is brown and white stand in the water', 'the large black dog and a small red and white dog are standing in the water', 'two dogs are sniffing each other while standing in a body of water', 'two dogs in the water']
o = 'a black and white dog is running through the water'
g = 'a dog and a cat playing in the water'
# d = d.split(' ')
# o = o.split(' ')
# g = g.split(' ')
print('GPT2 generated caption BLEU Score',nltk.translate.bleu_score.sentence_bleu(d, g,weights = (0.5,0.5)))
print('Our model generated caption BLEU Score',nltk.translate.bleu_score.sentence_bleu(d, o,weights = (0.5,0.5)))

GPT2 generated caption BLEU Score 0.9710083124552245
Our model generated caption BLEU Score 0.8453428098440103


In [16]:
end = time.time() #a man standing next to another man holding a cell phone

In [18]:
end-start

7.45050048828125

In [19]:
(end-start)*1000/60


124.17500813802083

In [21]:
from transformers import *

C:\Users\sauar\anaconda3\lib\site-packages\torchaudio\backend\utils.py:62: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [22]:
model = PegasusForConditionalGeneration.from_pretrained("tuner007/pegasus_paraphrase")
tokenizer = PegasusTokenizerFast.from_pretrained("tuner007/pegasus_paraphrase")

loading configuration file config.json from cache at C:\Users\sauar/.cache\huggingface\hub\models--tuner007--pegasus_paraphrase\snapshots\0159e2949ca73657a2f1329898f51b7bb53b9ab2\config.json
Model config PegasusConfig {
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 1,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
 

In [23]:
def get_paraphrased_sentences(model, tokenizer, sentence, num_return_sequences=5, num_beams=5):
  # tokenize the text to be form of a list of token IDs
  inputs = tokenizer([sentence], truncation=True, padding="longest", return_tensors="pt")
  # generate the paraphrased sentences
  outputs = model.generate(
    **inputs,
    num_beams=num_beams,
    num_return_sequences=num_return_sequences,
  )
  # decode the generated sentences using the tokenizer to get them back to text
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [24]:
sentence = "Learning is the process of acquiring new understanding, knowledge, behaviors, skills, values, attitudes, and preferences."
get_paraphrased_sentences(model, tokenizer, sentence, num_beams=10, num_return_sequences=5)

C:\Users\sauar\anaconda3\lib\site-packages\transformers\generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 60 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['Learning involves the acquisition of new understanding, knowledge, behaviors, skills, values, attitudes, and preferences.',
 'Learning is the acquisition of new understanding, knowledge, behaviors, skills, values, attitudes, and preferences.',
 'The process of learning is the acquisition of new understanding, knowledge, behaviors, skills, values, attitudes, and preferences.',
 'Gaining new understanding, knowledge, behaviors, skills, values, attitudes, and preferences is the process of learning.',
 'New understanding, knowledge, behaviors, skills, values, attitudes, and preferences are acquired through learning.']

In [25]:
import pandas as pd

In [26]:
df  =pd.read_excel('Captions.xlsx')

In [35]:
lis = df['Caption 1']

In [37]:
lis = lis.dropna()

In [38]:
lis = lis.tolist()

In [39]:
cap1 = []
cap2 = []
cap3= []
cap4 = []
cap5 = []
for i in lis:
    n = get_paraphrased_sentences(model, tokenizer, i, num_beams=10, num_return_sequences=5)
    cap1.append(n[0])
    cap2.append(n[1])
    cap3.append(n[2])
    cap4.append(n[3])
    cap5.append(n[4])

C:\Users\sauar\anaconda3\lib\site-packages\transformers\generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 60 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['A group of men are attacking each other.',
 'A masked man is walking.',
 'A man is fighting another man.',
 'A group of men are attacking each other.',
 'A man is harrasssing a woman.',
 'A man is pointing a gun.',
 'A man is near a car.',
 'A man is harrasssing a woman.',
 'A man is fighting another man.',
 'A man is wearing a hoodie.',
 'A group of men are attacking each other.',
 'A man wearing a black shirt is attacking another man.',
 'A group of men are attacking each other.',
 'A group of men are attacking each other.',
 'A masked man is holding a gun.',
 'A man is abducting a boy.',
 'A masked man is abducting a girl.',
 'A man is abducting a baby.',
 'A masked man is abducting a woman.',
 'A man with a baseball bat is wearing a black hoodie.',
 'A man with a baseball bat is wearing a black hoodie.',
 'A man with a baseball bat is wearing a black hoodie.',
 'A man with a baseball bat is wearing a white shirt.',
 'A masked man is holding a baseball bat.',
 'A man with a baseba